In [3]:
import h2o
from h2o.automl import H2OAutoML

# Start the H2O cluster (locally)
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.301-b09, mixed mode)
  Starting server from C:\Users\mwils\Documents\Python\ML\venv\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\mwils\AppData\Local\Temp\tmpikctenux
  JVM stdout: C:\Users\mwils\AppData\Local\Temp\tmpikctenux\h2o_mwils_started_from_python.out
  JVM stderr: C:\Users\mwils\AppData\Local\Temp\tmpikctenux\h2o_mwils_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,America/Santiago
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.2
H2O_cluster_version_age:,1 month and 14 days
H2O_cluster_name:,H2O_from_python_mwils_47jp78
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.511 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [15]:
# Import a sample binary outcome train/test set into H2O
train = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_train_10k.csv")
test = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_test_5k.csv")

# Identify predictors and response
x = train.columns
y = "response"
x.remove(y)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [52]:
train.head()

response,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28
1,0.869293,-0.635082,0.22569,0.32747,-0.689993,0.754202,-0.248573,-1.09206,0,1.37499,-0.653674,0.930349,1.10744,1.1389,-1.5782,-1.04699,0,0.65793,-0.0104546,-0.0457672,3.10196,1.35376,0.979563,0.978076,0.920005,0.721657,0.988751,0.876678
1,0.907542,0.329147,0.359412,1.49797,-0.31301,1.09553,-0.557525,-1.58823,2.17308,0.812581,-0.213642,1.27101,2.21487,0.499994,-1.26143,0.732156,0,0.398701,-1.13893,-0.00081911,0,0.30222,0.833048,0.9857,0.978098,0.779732,0.992356,0.798343
1,0.798835,1.47064,-1.63597,0.453773,0.425629,1.10487,1.28232,1.38166,0,0.851737,1.54066,-0.81969,2.21487,0.99349,0.35608,-0.208778,2.54822,1.25695,1.12885,0.900461,0,0.909753,1.10833,0.985692,0.951331,0.803252,0.865924,0.780118
0,1.34438,-0.876626,0.935913,1.99205,0.882454,1.78607,-1.64678,-0.942383,0,2.42326,-0.676016,0.736159,2.21487,1.29872,-1.43074,-0.364658,0,0.745313,-0.678379,-1.36036,0,0.946652,1.0287,0.998656,0.728281,0.8692,1.02674,0.957904
1,1.10501,0.321356,1.5224,0.882808,-1.20535,0.681466,-1.07046,-0.921871,0,0.800872,1.02097,0.971407,2.21487,0.596761,-0.350273,0.631194,0,0.479999,-0.373566,0.113041,0,0.755856,1.36106,0.98661,0.838085,1.1333,0.872245,0.808487
0,1.59584,-0.607811,0.00707492,1.81845,-0.111906,0.84755,-0.566437,1.58124,2.17308,0.755421,0.64311,1.42637,0,0.921661,-1.19043,-1.61559,0,0.651114,-0.654227,-1.27434,3.10196,0.823761,0.938191,0.971758,0.789176,0.430553,0.961357,0.957818
1,0.409391,-1.88468,-1.02729,1.67245,-1.6046,1.33801,0.0554274,0.0134659,2.17308,0.509783,-1.03834,0.707862,0,0.746918,-0.358465,-1.64665,0,0.367058,0.0694965,1.37713,3.10196,0.869418,1.22208,1.00063,0.545045,0.698653,0.977314,0.828786
1,0.933895,0.62913,0.527535,0.238033,-0.966569,0.547811,-0.0594392,-1.70687,2.17308,0.941003,-2.65373,-0.15722,0,1.03037,-0.175505,0.523021,2.54822,1.37355,1.29125,-1.46745,0,0.901837,1.08367,0.979696,0.7833,0.849195,0.894356,0.774879
1,1.40514,0.536603,0.689554,1.17957,-0.110061,3.2024,-1.52696,-1.57603,0,2.93154,0.567342,-0.130033,2.21487,1.78712,0.899499,0.585151,2.54822,0.401865,-0.151202,1.16349,0,1.66707,4.03927,1.17583,1.04535,1.54297,3.53483,2.74075
1,1.17657,0.104161,1.397,0.479721,0.265513,1.13556,1.53483,-0.253291,0,1.02725,0.534316,1.18002,0,2.40566,0.0875568,-0.976534,2.54822,1.25038,0.268541,0.530334,0,0.833175,0.773968,0.98575,1.1037,0.84914,0.937104,0.812364


In [16]:
# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [8]:
# Run AutoML for 20 base models
aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |
13:22:06.761: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_1_AutoML_1_20220705_132206

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.09527945529839002
RMSE: 0.3086737036068833
LogLoss: 0.33626129492580287
Null degrees of freedom: 9999
Residual degrees of freedom: 9987
Null deviance: 13828.11338742424
Residual deviance: 6725.225898516058
AIC: 6751.225898516058
AUC: 0.9702984890403494
AUCPR: 0.9734166658450707
Gini: 0.9405969780806989

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.47826848382518183: 


,,0,1,Error,Rate
0,0,4083.0,622.0,0.1322,(622.0/4705.0)
1,1,329.0,4966.0,0.0621,(329.0/5295.0)
2,Total,4412.0,5588.0,0.0951,(951.0/10000.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.478268,0.912616,210.0
1,max f2,0.373882,0.944825,250.0
2,max f0point5,0.597702,0.923373,166.0
3,max accuracy,0.478268,0.904900,210.0
4,max precision,0.995458,1.000000,0.0
5,max recall,0.110328,1.000000,360.0
6,max specificity,0.995458,1.000000,0.0
7,max absolute_mcc,0.478268,0.809882,210.0
8,max min_per_class_accuracy,0.523365,0.900744,194.0
9,max mean_per_class_accuracy,0.478268,0.902833,210.0



Gains/Lift Table: Avg response rate: 52,95 %, avg score: 52,67 %


...


Gains/Lift Table: Avg response rate: 52,95 %, avg score: 52,95 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.01,0.947516,1.869688,1.869688,0.990,0.961527,0.990000,0.961527,0.018697,0.018697,86.968839,86.968839,0.018484
1,2,0.02,0.934129,1.756374,1.813031,0.930,0.939597,0.960000,0.950562,0.017564,0.036261,75.637394,81.303116,0.034560
2,3,0.03,0.921143,1.813031,1.813031,0.960,0.927018,0.960000,0.942714,0.018130,0.054391,81.303116,81.303116,0.051840
3,4,0.04,0.913198,1.756374,1.798867,0.930,0.917156,0.952500,0.936325,0.017564,0.071955,75.637394,79.886686,0.067916
4,5,0.05,0.903933,1.775260,1.794145,0.940,0.908672,0.950000,0.930794,0.017753,0.089707,77.525968,79.414542,0.084394
5,6,0.10,0.867124,1.714825,1.754485,0.908,0.885292,0.929000,0.908043,0.085741,0.175449,71.482531,75.448536,0.160358
6,7,0.15,0.827451,1.620397,1.709789,0.858,0.847454,0.905333,0.887847,0.081020,0.256468,62.039660,70.978911,0.226288
7,8,0.20,0.789010,1.525968,1.663834,0.808,0.808510,0.881000,0.868012,0.076298,0.332767,52.596789,66.383381,0.282182
8,9,0.30,0.707297,1.405099,1.577589,0.744,0.748036,0.835333,0.828020,0.140510,0.473277,40.509915,57.758892,0.368282
9,10,0.40,0.619995,1.218130,1.487724,0.645,0.664063,0.787750,0.787031,0.121813,0.595090,21.813031,48.772427,0.414643




Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.699026,0.008625,0.698925,0.710825,0.686523,0.700348,0.698506
1,auc,0.791494,0.007767,0.797777,0.799260,0.780397,0.787367,0.792667
2,err,0.300974,0.008625,0.301075,0.289175,0.313477,0.299652,0.301494
3,err_count,602.000000,29.385370,616.000000,585.000000,642.000000,602.000000,565.000000
4,f0point5,0.697477,0.009675,0.698963,0.704030,0.680487,0.701442,0.702466
5,f1,0.754298,0.011838,0.761610,0.757161,0.733389,0.758233,0.761099
6,f2,0.821242,0.015945,0.836592,0.818966,0.795209,0.825031,0.830411
7,lift_top_group,1.870895,0.069565,1.861692,1.921178,1.954198,1.776540,1.840864
8,logloss,0.547885,0.008911,0.540331,0.540836,0.560739,0.553454,0.544065
9,max_per_class_error,0.497837,0.030160,0.529039,0.457732,0.477000,0.503219,0.522196



See the whole table with table.as_data_frame()


In [38]:
# Best model
aml.get_best_model()

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_1_AutoML_1_20220705_132206

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.09527945529839002
RMSE: 0.3086737036068833
LogLoss: 0.33626129492580287
Null degrees of freedom: 9999
Residual degrees of freedom: 9987
Null deviance: 13828.11338742424
Residual deviance: 6725.225898516058
AIC: 6751.225898516058
AUC: 0.9702984890403494
AUCPR: 0.9734166658450707
Gini: 0.9405969780806989

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.47826848382518183: 


,,0,1,Error,Rate
0,0,4083.0,622.0,0.1322,(622.0/4705.0)
1,1,329.0,4966.0,0.0621,(329.0/5295.0)
2,Total,4412.0,5588.0,0.0951,(951.0/10000.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.478268,0.912616,210.0
1,max f2,0.373882,0.944825,250.0
2,max f0point5,0.597702,0.923373,166.0
3,max accuracy,0.478268,0.904900,210.0
4,max precision,0.995458,1.000000,0.0
5,max recall,0.110328,1.000000,360.0
6,max specificity,0.995458,1.000000,0.0
7,max absolute_mcc,0.478268,0.809882,210.0
8,max min_per_class_accuracy,0.523365,0.900744,194.0
9,max mean_per_class_accuracy,0.478268,0.902833,210.0



Gains/Lift Table: Avg response rate: 52,95 %, avg score: 52,67 %


...


Gains/Lift Table: Avg response rate: 52,95 %, avg score: 52,95 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.01,0.947516,1.869688,1.869688,0.990,0.961527,0.990000,0.961527,0.018697,0.018697,86.968839,86.968839,0.018484
1,2,0.02,0.934129,1.756374,1.813031,0.930,0.939597,0.960000,0.950562,0.017564,0.036261,75.637394,81.303116,0.034560
2,3,0.03,0.921143,1.813031,1.813031,0.960,0.927018,0.960000,0.942714,0.018130,0.054391,81.303116,81.303116,0.051840
3,4,0.04,0.913198,1.756374,1.798867,0.930,0.917156,0.952500,0.936325,0.017564,0.071955,75.637394,79.886686,0.067916
4,5,0.05,0.903933,1.775260,1.794145,0.940,0.908672,0.950000,0.930794,0.017753,0.089707,77.525968,79.414542,0.084394
5,6,0.10,0.867124,1.714825,1.754485,0.908,0.885292,0.929000,0.908043,0.085741,0.175449,71.482531,75.448536,0.160358
6,7,0.15,0.827451,1.620397,1.709789,0.858,0.847454,0.905333,0.887847,0.081020,0.256468,62.039660,70.978911,0.226288
7,8,0.20,0.789010,1.525968,1.663834,0.808,0.808510,0.881000,0.868012,0.076298,0.332767,52.596789,66.383381,0.282182
8,9,0.30,0.707297,1.405099,1.577589,0.744,0.748036,0.835333,0.828020,0.140510,0.473277,40.509915,57.758892,0.368282
9,10,0.40,0.619995,1.218130,1.487724,0.645,0.664063,0.787750,0.787031,0.121813,0.595090,21.813031,48.772427,0.414643




Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.699026,0.008625,0.698925,0.710825,0.686523,0.700348,0.698506
1,auc,0.791494,0.007767,0.797777,0.799260,0.780397,0.787367,0.792667
2,err,0.300974,0.008625,0.301075,0.289175,0.313477,0.299652,0.301494
3,err_count,602.000000,29.385370,616.000000,585.000000,642.000000,602.000000,565.000000
4,f0point5,0.697477,0.009675,0.698963,0.704030,0.680487,0.701442,0.702466
5,f1,0.754298,0.011838,0.761610,0.757161,0.733389,0.758233,0.761099
6,f2,0.821242,0.015945,0.836592,0.818966,0.795209,0.825031,0.830411
7,lift_top_group,1.870895,0.069565,1.861692,1.921178,1.954198,1.776540,1.840864
8,logloss,0.547885,0.008911,0.540331,0.540836,0.560739,0.553454,0.544065
9,max_per_class_error,0.497837,0.030160,0.529039,0.457732,0.477000,0.503219,0.522196



See the whole table with table.as_data_frame()


In [41]:
# Optional
# Save Best model
model_path = h2o.save_model(model=aml.get_best_model(), path=os.getcwdb().decode(), force=True)

# NOTE
# i can specificate algorithm and criterion for get_best_model. 
# Example: get_best_model(algorithm="xgboost", criterion="logloss") 


In [13]:
# Optional
# Read Best model
model_path = os.getcwdb().decode()+os.sep+'StackedEnsemble_AllModels_1_AutoML_1_20220705_132206'
saved_model = h2o.load_model(model_path)

In [43]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_1_AutoML_1_20220705_132206,0.791088,0.54797,0.808649,0.314158,0.430999,0.18576
StackedEnsemble_BestOfFamily_1_AutoML_1_20220705_132206,0.788103,0.551646,0.805756,0.324138,0.432571,0.187117
GBM_5_AutoML_1_20220705_132206,0.7829,0.558001,0.802101,0.336502,0.435272,0.189462
GBM_1_AutoML_1_20220705_132206,0.779512,0.560256,0.799536,0.327511,0.436609,0.190627
GBM_2_AutoML_1_20220705_132206,0.778879,0.56126,0.797195,0.329521,0.43698,0.190952
GBM_grid_1_AutoML_1_20220705_132206_model_2,0.77786,0.564655,0.795359,0.33376,0.438088,0.191921
GBM_3_AutoML_1_20220705_132206,0.776908,0.563372,0.793988,0.321192,0.437927,0.19178
GBM_grid_1_AutoML_1_20220705_132206_model_3,0.772926,0.568181,0.791195,0.322808,0.439997,0.193598
GBM_4_AutoML_1_20220705_132206,0.771118,0.569616,0.791326,0.33599,0.441149,0.194612
GBM_grid_1_AutoML_1_20220705_132206_model_4,0.770522,0.569244,0.7891,0.349737,0.440792,0.194297


In [18]:
# Predict new values in best model
preds = saved_model.predict(test)
print(preds.head())

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


predict,p0,p1
0,0.719821,0.280179
0,0.692308,0.307692
1,0.500479,0.499521
1,0.346992,0.653008
0,0.685265,0.314735
1,0.228062,0.771938
1,0.251963,0.748037
1,0.578459,0.421541
1,0.527178,0.472822
0,0.850797,0.149203


In [50]:
saved_model.model_performance(test)


ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.18426201497125116
RMSE: 0.42925751591702055
LogLoss: 0.5456266441694188
Null degrees of freedom: 4999
Residual degrees of freedom: 4987
Null deviance: 6905.1964032609685
Residual deviance: 5456.266441694188
AIC: 5482.266441694188
AUC: 0.794579034794535
AUCPR: 0.8134330866511048
Gini: 0.5891580695890699

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.32660244308148156: 


,,0,1,Error,Rate
0,0,1033.0,1282.0,0.5538,(1282.0/2315.0)
1,1,250.0,2435.0,0.0931,(250.0/2685.0)
2,Total,1283.0,3717.0,0.3064,(1532.0/5000.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.326602,0.760700,279.0
1,max f2,0.145423,0.864370,354.0
2,max f0point5,0.589853,0.753096,161.0
3,max accuracy,0.515446,0.723200,194.0
4,max precision,0.943329,0.983607,9.0
5,max recall,0.017151,1.000000,397.0
6,max specificity,0.986036,0.999568,0.0
7,max absolute_mcc,0.515446,0.446797,194.0
8,max min_per_class_accuracy,0.507936,0.721043,198.0
9,max mean_per_class_accuracy,0.515446,0.723997,194.0



Gains/Lift Table: Avg response rate: 53,70 %, avg score: 52,04 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.01,0.945807,1.824953,1.824953,0.980,0.958832,0.980000,0.958832,0.018250,0.018250,82.495345,82.495345,0.017818
1,2,0.02,0.931704,1.787709,1.806331,0.960,0.937643,0.970000,0.948238,0.017877,0.036127,78.770950,80.633147,0.034831
2,3,0.03,0.922380,1.713222,1.775295,0.920,0.926862,0.953333,0.941113,0.017132,0.053259,71.322160,77.529485,0.050235
3,4,0.04,0.911709,1.787709,1.778399,0.960,0.917466,0.955000,0.935201,0.017877,0.071136,78.770950,77.839851,0.067248
4,5,0.05,0.902156,1.713222,1.765363,0.920,0.906330,0.948000,0.929427,0.017132,0.088268,71.322160,76.536313,0.082653
5,6,0.10,0.861331,1.720670,1.743017,0.924,0.882624,0.936000,0.906025,0.086034,0.174302,72.067039,74.301676,0.160479
6,7,0.15,0.818580,1.512104,1.666046,0.812,0.840625,0.894667,0.884225,0.075605,0.249907,51.210428,66.604593,0.215782
7,8,0.20,0.776466,1.608939,1.651769,0.864,0.797625,0.887000,0.862575,0.080447,0.330354,60.893855,65.176909,0.281542
8,9,0.30,0.693773,1.363128,1.555556,0.732,0.736157,0.835333,0.820436,0.136313,0.466667,36.312849,55.555556,0.359971
9,10,0.40,0.603458,1.292365,1.489758,0.694,0.649754,0.800000,0.777765,0.129236,0.595903,29.236499,48.975791,0.423117


NameError: name 'preds' is not defined